# 0. 환경조성

## (1) 라이브러리와 데이터 준비

In [ ]:
# mecab 설치
$ sudo apt-get install curl git
$ bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

import re
import os
import io
import time
import random

from sklearn.model_selection import train_test_split

print(tf.__version__)


2.4.1


In [3]:
from konlpy.tag import Mecab
import gensim   # 버전 4.x
from gensim.models import Word2Vec
print(gensim.__version__)

4.0.1


/opt/conda/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


# Step1. 데이터 다운로드
- songys/Chatbot_data  https://github.com/songys/Chatbot_data
- ChatbotData.csv 를 다운로드해 챗봇 훈련 데이터를 확보
- pandas 라이브러리로 .csv파일 읽음
- 읽어 온 데이터의 질문과 답변을 각각 questions, anwers 변수에 나눠서 저장
- 심볼릭 링크 생성하면 데이터를 다운로드 할 필요가 없다.

$wget https://github.com/songys/Chatbot_data

$mv ChatbotData\ .csv ~/aiffel/transformer_chatbot

In [4]:
path_to_file = os.getenv('HOME') + '/aiffel/NLP/12_transformer_chatbot/ChatbotData .csv'
data = pd.read_csv(path_to_file, encoding='UTF-8')
data

#for sen in corpus[0:100][::20]: print('>>', sen)

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2


In [5]:
# Q데이터 A데이터 저장하기
src = []
tgt = []
for s,t in zip(data['Q'],data['A']):
    src.append(str(s))
    tgt.append(str(t))

In [6]:
len(tgt)

11823

In [7]:
src[0]

'12시 땡!'

In [8]:
len(src)

11823

In [9]:
tgt[0]

'하루가 또 가네요.'

In [10]:
len(tgt)

11823

# Step2. 데이터 정제

### preprocess_sentence() 함수
1. 영문자의 경우, 모두 소문자로 변환한다.
2. 영문자와 한글, 숫자, 그리고 주요 특수문자를 제외하곤 정규식을 활용하여 모두 제거한다.

In [11]:
# from konlpy.tag import Mecab
# mecab = Mecab()

def preprocess_sentence(sentence):
    from konlpy.tag import Mecab
    mecab = Mecab()
    
    sentence = sentence.lower().strip()

    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^0-9ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z?.!,]+", " ", sentence)

    sentence = sentence.strip()
    corpus = mecab.morphs(sentence)
    
    return corpus

In [12]:
print(preprocess_sentence('소스 문장 데이터와 타겟 문장 데이터를 입력으로 받습니다.'))

['소스', '문장', '데이터', '와', '타', '겟', '문장', '데이터', '를', '입력', '으로', '받', '습니다', '.']


In [13]:
print(preprocess_sentence(src[0]))

['12', '시', '땡', '!']


In [14]:
print(preprocess_sentence(tgt[0]))

['하루', '가', '또', '가', '네요', '.']


# Step3. 데이터 토큰화

- 토큰화에는 KoNLPy의 mecab 클래스를 사용한다

## build_corpus함수 생성
1. 소스 문장 데이터와 타겟 문장 데이터를 입력으로 받는다.
2. 데이터를 앞서 정의한 preprocess_sentence() 함수로 정제하고, 토큰화한다.
3. 토큰화는 전달받은 토큰나이즈 함수를 사용한다.
   - mecab.morphs 함수를 전달
4. 토큰의 갯수가 일정 길이 이상인 문장은 데이터에서 제외한다.
5. 중복되는 문장은 데이터에서 제외한다.
   - 소스:타겟 쌍을 비교하지 않고 소스는 소스대로 타겟은 타겟대로 검사
     - 중복쌍이 흐트러지지 않도록 유의
     

- 구현한 함수를 활용하여 questions와 answers를 각각 que_corpus, ans_corpus에 토큰화하여 저장 

In [15]:
cleaned_corpus = list(set(zip(src,tgt)))

In [16]:
# 데이터 정제
src_corpus = []
tgt_corpus = []


for tmp in cleaned_corpus:
    #print(tmp[0])
    #print(tmp[1])
    tmp_src = preprocess_sentence(tmp[0])
    tmp_tgt = preprocess_sentence(tmp[1])
    #if len(tmp_ko) <= 40:
    src_corpus.append(tmp_src)
    tgt_corpus.append(tmp_tgt)

    

print(len(src_corpus))
print(len(tgt_corpus))
print("Questions", src_corpus[100])   
print("Answers:", tgt_corpus[100])

11750
11750
Questions ['정성', '담긴', '선물', '뭐', '가', '좋', '을까', '?']
Answers: ['손', '편지', '도', '좋', '을', '거', '같', '아요', '.']


In [17]:
src_corpus[:10]

[['헤어진', '여자', '친구', '와', '재회', '하', '고', '싶', '어'],
 ['내일', '뭐', '입', '고', '나갈까'],
 ['쉬운', '게', '하나', '도', '없', '는', '듯'],
 ['내', '사랑', '이', '있', '을까', '?'],
 ['내', '가', '생각', '했', '던', '사람', '이', '맞', '는지', '확신', '이', '안', '들', '어'],
 ['다른', '사람', '들', '도', '이런', '재회', '바라', '나', '?'],
 ['썸', '타', '는', '중', '인데', '다른', '사람', '과', '연락', '자제', '해야', '해', '?'],
 ['드디어', '그', '순간', '이', '거의', '다가왔', '습니다', '.'],
 ['다가오', '는', '크리', '마스'],
 ['환기', '좀', '해야', '할까', '?']]

In [18]:
tgt_corpus[:10]

[['후회', '하', '지', '않', '을', '자신', '이', '있', '으면', '연락', '해', '보', '세요', '.'],
 ['날씨', '에', '맞', '게', '입', '는', '게', '좋', '을', '것', '같', '아요', '.'],
 ['저', '도', '어려운', '게', '투성이', '에요', '.'],
 ['눈', '을', '크', '게', '뜨', '고', '잘', '찾아보', '세요', '.'],
 ['새로운', '모습', '을', '봤', '나', '봐요', '.'],
 ['재회', '는', '한', '번', '쯤', '꿈', '꿎'],
 ['잘',
  '되',
  '고',
  '싶',
  '다면',
  '자제',
  '하',
  '는',
  '것',
  '도',
  '한',
  '방법',
  '이',
  '겠',
  '죠',
  '.'],
 ['마지막', '이', '온', '걸까요', '.'],
 ['빨간', '날', '일', '뿐', '이', '죠', '.'],
 ['매일', '환기', '하', '는', '게', '좋', '대요', '.']]

In [19]:
que_corpus = src_corpus
ans_corpus = tgt_corpus

# Step4. Augmentation
- 1만 개 가량 데이터 => Lexical Substitution 함수를 적용하여 데이터 확대
- 한국어 事前 훈련된 Embedding 모델을 다운로드
   - Kyubyong/wordvectors  https://github.com/Kyubyong/wordvectors
     - Korean(w) : Word2Vec으로 학습한 모델
     - Korean(w)를 다운로드하고, ko.bin 파일을 확보 
- Augmentation된 que_corpus 와 원본 ans_corpus 가 병렬을 이루도록, 이후엔 반대로 원본 que_corpus 와 Augmentation된 ans_corpus 가 병렬을 이루도록 하여 전체 데이터가 원래의 3배가량으로 늘어나도록 합니다.


In [20]:
!pip list | grep gensim

gensim                        4.0.1
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [21]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

#word2vec_path = os.getenv('HOME') + '/aiffel/NLP/12_transformer_chatbot/ko.bin'
#word2vec = Word2Vec.load(word2vec_path)  
word2vec_path = os.getenv('HOME') + '/aiffel/NLP/12_transformer_chatbot/wiki.ko.vec'

word2vec = gensim.models.KeyedVectors.load_word2vec_format(word2vec_path)  


In [22]:
word2vec.similar_by_word("짜증")

[('짜증날', 0.8327823281288147),
 ('짜증나', 0.8251842260360718),
 ('짜증나는', 0.8198283314704895),
 ('짜증나고', 0.806260347366333),
 ('짜증난다', 0.795647144317627),
 ('짜증내는', 0.788124680519104),
 ('짜증나서', 0.784796953201294),
 ('짜증이', 0.7729315161705017),
 ('짜증나게', 0.7662945985794067),
 ('싫증', 0.7639327645301819)]

In [23]:
word2vec.similar_by_word("분노")

[('분노나', 0.7223528027534485),
 ('분노도', 0.6828715801239014),
 ('분노에', 0.6590042114257812),
 ('분노와', 0.6552071571350098),
 ('분노감을', 0.6441459059715271),
 ('분노로', 0.6439695358276367),
 ('분노는', 0.6429007053375244),
 ('분노할', 0.641103208065033),
 ('분노염을', 0.6329343318939209),
 ('분노하는', 0.6321898698806763)]

## Lexical Substitution 구현하기
- 입력된 문장을 Embedding 유사도를 기반으로 Augmentation하여 반환하는 lexical_sub()을 구현
- 구현한 함수를 활용해 3,000개의 영문 데이터를 Augmentation하고 결과를 확인
- 단어장에 포함되지 않은 단어가 들어오는 경우, 문장 부호에 대한 치환이 발생하는 경우 등의 예외는 자유롭게 처리

In [24]:
# Lexical Substitution 구현하기
def lexical_sub(sentence, word2vec):
    import random

    res = ""
    toks = sentence

    try:
        _from = random.choice(toks)
        _to = word2vec.most_similar(_from)[0][0]

    except:   # 단어장에 없는 단어|
        return None

    for tok in toks:
        if tok is _from: res += _to + " "
        else: res += tok + " "

    return res

In [25]:
print(que_corpus[100])
print(lexical_sub(que_corpus[1], word2vec))

['정성', '담긴', '선물', '뭐', '가', '좋', '을까', '?']
내일쯤 뭐 입 고 나갈까 


In [26]:
from tqdm import tqdm_notebook

new_que_corpus = []
new_ans_corpus = []

# Augmentation된 que_corpus 와 원본 ans_corpus 가 병렬을 이루도록
for idx in tqdm_notebook(range(len(que_corpus))):
    que_augmented = lexical_sub(que_corpus[idx], word2vec)
    ans = ans_corpus[idx]
    
    if que_augmented is not None:
        new_que_corpus.append(que_augmented.split())
        new_ans_corpus.append(ans)
        
    else:
       
        continue
    
for idx in tqdm_notebook(range(len(ans_corpus))):
    que = que_corpus[idx]
    ans_augmented = lexical_sub(ans_corpus[idx], word2vec)
    
    if ans_augmented is not None:
        new_que_corpus.append(que)
        new_ans_corpus.append(ans_augmented.split())
       
    else:
       
        continue

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


  0%|          | 0/11750 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/11750 [00:00<?, ?it/s]

In [27]:
print(new_que_corpus[:10])
print(new_ans_corpus[:10])

[['헤어진', '여자', '친구', '와', '재회', '하', '고', '싶', '어어'], ['내일', '뭐', '입과', '고', '나갈까'], ['쉬운', '것도', '하나', '도', '없', '는', '듯'], ['내', '사랑', '그', '있', '을까', '?'], ['다른', '사람', '들', '도', '이런', '재회', '바라', '와', '?'], ['썸', '타', '는', '중', '인데', '다른', '사람', '와', '연락', '자제', '해야', '해', '?'], ['드디어', '이', '순간', '이', '거의', '다가왔', '습니다', '.'], ['다가오', '는', '크리', '테라포마스'], ['환기', '좀', '해야', '할까', '나요'], ['길', '이', '안되도', '보여']]
[['후회', '하', '지', '않', '을', '자신', '이', '있', '으면', '연락', '해', '보', '세요', '.'], ['날씨', '에', '맞', '게', '입', '는', '게', '좋', '을', '것', '같', '아요', '.'], ['저', '도', '어려운', '게', '투성이', '에요', '.'], ['눈', '을', '크', '게', '뜨', '고', '잘', '찾아보', '세요', '.'], ['재회', '는', '한', '번', '쯤', '꿈', '꿎'], ['잘', '되', '고', '싶', '다면', '자제', '하', '는', '것', '도', '한', '방법', '이', '겠', '죠', '.'], ['마지막', '이', '온', '걸까요', '.'], ['빨간', '날', '일', '뿐', '이', '죠', '.'], ['매일', '환기', '하', '는', '게', '좋', '대요', '.'], ['너무', '낙담', '하', '지', '마세요', '.']]


In [28]:
print(len(new_que_corpus))
print(len(new_ans_corpus))

21798
21798


In [29]:
que_corpus = que_corpus + new_que_corpus
print(len(que_corpus))
ans_corpus = ans_corpus + new_ans_corpus
print(len(ans_corpus))

33548
33548


In [30]:
que_corpus[20400]

['헤어진지', '두', '달', '늘', '만났', '던', '장소는', '를', '.', '난', '지나다니', '고']

In [31]:
ans_corpus[20400]

['무덤덤', '해', '지', '길', '바랄게요', '.']

# Step5. 데이터 벡터화
- 타겟 데이터인 ans_corpus 에 \<start> 토큰과 \<end> 토큰이 추가되지 않은 상태이니 이를 먼저 해결한 후 벡터화를 진행합니다. 
- 우리가 구축한 ans_corpus 는 list 형태임을 활용

1. 타겟 데이터 전체에 \<start>와 \<end>토큰을 추가한다.
   - 챗봇 훈련 데이터의 가장 큰 특징 중 하나는 소스 데이터와 타겟 데이터가 같은 언어를 사용한다는 것임
   - Embedding층을 공유할 수 있음
   
2. 특수 토큰을 더함으로써 ans_corpus 또한 완성이 되었으니, que_corpus와 결합하여 전체 데이터에 대한 단어 사전을 구축하고 벡터화하여 enc_train과 dec_train을 확보한다.

In [32]:
tgt_corpus = []

for corpus in ans_corpus:
    tgt_corpus.append(["<start>"] + corpus + ["<end>"])
    
print(tgt_corpus[0])
print(tgt_corpus[325])
print(tgt_corpus[395])
ans_corpus = tgt_corpus

['<start>', '후회', '하', '지', '않', '을', '자신', '이', '있', '으면', '연락', '해', '보', '세요', '.', '<end>']
['<start>', '실수', '할', '수', '도', '있', '지요', '.', '<end>']
['<start>', '그분', '의', '관심사', '에', '대한', '얘기', '가', '좋', '겠', '네요', '.', '<end>']


In [33]:
from collections import Counter

voc_data = que_corpus + ans_corpus

words = np.concatenate(voc_data).tolist()
counter = Counter(words)
counter = counter.most_common(30000-2)
vocab = ['<pad>', '<unk>'] + [key for key, _ in counter]
word_to_index = {word:index for index, word in enumerate(vocab)}
index_to_word = {index:word for word, index in word_to_index.items()}

In [34]:
# # 벡터화
def get_encoded_sentence(sentence, word_to_index):
    return [word_to_index[word] if word in word_to_index else word_to_index['<unk>'] for word in sentence]

def get_decoded_sentence(encoded_sentence, index_to_word):
    return ' '.join(index_to_word[index] if index in index_to_word else '<unk>' for index in encoded_sentence[1:])  #[1:]를 통해 <BOS>를 제외

def vectorize(corpus, word_to_index):
    data = []
    for sen in corpus:
        sen = get_encoded_sentence(sen, word_to_index)
        data.append(sen)
    return data

que_train = vectorize(que_corpus, word_to_index)
ans_train = vectorize(ans_corpus, word_to_index)

print(len(que_train))
print(len(ans_train))

33548
33548


In [35]:
# # 패팅처리
enc_tensor = tf.keras.preprocessing.sequence.pad_sequences(que_train, padding='post')
dec_tensor = tf.keras.preprocessing.sequence.pad_sequences(ans_train, padding='post')

enc_train, enc_val, dec_train, dec_val = \
train_test_split(enc_tensor, dec_tensor, test_size=0.01) # test set은 1%만

print(len(enc_train))
print(len(enc_val)) 
print(len(dec_train))
print(len(dec_val))

33212
336
33212
336


In [36]:
enc_train[0]

array([ 646, 1807,  157,  724,   20,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
      dtype=int32)

In [37]:
dec_train[0]

array([  3, 856, 814, 174, 157, 574,  10,   2,   4,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0], dtype=int32)

In [38]:
print(len(enc_train[0]))
print(len(dec_train[0]))

32
42


In [ ]:
# def generate_tokenizer(corpus,
#                       vocab_size,
#                       lang='spa-eng',
#                       pad_id=0,    # pad token의 일련번호
#                       bos_id=1,    # 문장의 시작을 의미하는 bos token
#                       eos_id=2,    # 문장의 끝을 의미하는 eos token
#                       unk_id=3):   # 모르는 단어 unk token
#     file = f'./{lang}_corpus.txt' 
#     model = f'{lang}_spm'
    
#     with open(file, 'w', encoding='UTF-8') as f:
#         for row in corpus: f.write(str(row) + '\n')
            
#     import sentencepiece as spm
#     spm.SentencePieceTrainer.Train(
#         f'--input=./{file} --model_prefix={model} --vocab_size={vocab_size}' + 
#         f'--pad_id=={pad_id} --bos_id={bos_id} --eos_id={eos_id} --unk_id={unk_id}'
#     )
    
#     tokenizer = spm.SentencePieceProcessor()
#     tokenizer.Load(f'{model}.model')
    
#     return tokenizer
# print('슝=3')

In [ ]:
# # 중복 단어 제거
# src_corpus = [tuple(l) for l in src_corpus]  # 리스트의 리스트로 튜플로 변경후 set으로 변환
# src_cleaned_corpus = set(src_corpus)
# src_cleaned_corpus = list(src_cleaned_corpus)


# VOCAB_SIZE = 2245
# tokenizer = generate_tokenizer(src_cleaned_corpus, VOCAB_SIZE)
# tokenizer.set_encode_extra_options('bos:eos') # 문장 양 끝에 <s>, </s>추가

In [ ]:
# # 패팅처리
# enc_tensor = tf.keras.preprocessing.sequence.pad_sequences(src_corpus, padding='post')
# dec_tensor = tf.keras.preprocessing.sequence.pad_sequences(tgt_corpus, padding='post')

# enc_train, enc_val, dec_train, dec_val = train_test_split(enc_tensor, dec_tensor,
#                                                          test_size=0.01)

# print(f'enc_train : {len(enc_train)}', f'    enc_val : {len(enc_val)}')
# print(f'dec_train : {len(dec_train)}', f'    dec_val : {len(dec_val)}')

# Step6. 훈련하기
- 앞서 번역 모델을 훈련하며 정의한 Transformer 를 그대로 사용한다.
- 대신 데이터의 크기가 작으니 하이퍼파라미터를 튜닝해야 과적합을 피할 수 있습니다. 
- 모델을 훈련하고 아래 예문에 대한 답변을 생성하세요! 가장 멋진 답변과 모델의 하이퍼파라미터를 제출하시면 됩니다.

\# 예문
1. 지루하다, 놀러가고 싶어.
2. 오늘 일찍 일어났더니 피곤하다.
3. 간만에 여자친구랑 데이트 하기로 했어.
4. 집에 있는다는 소리야.

---

\# 제출

Translations
> 1. 잠깐 쉬 어도 돼요 . <end>
> 2. 맛난 거 드세요 . <end>
> 3. 떨리 겠 죠 . <end>
> 4. 좋 아 하 면 그럴 수 있 어요 . <end>

Hyperparameters
> n_layers: 1
> d_model: 368
> n_heads: 8
> d_ff: 1024
> dropout: 0.2

Training Parameters
> Warmup Steps: 1000
> Batch Size: 64
> Epoch At: 10

## (1) 트랜스포머 구현

### 1) Positional Encoding

In [39]:
def positional_encoding(pos, d_model):
    def cal_angle(position, i):
        return position / np.power(10000, int(i) / d_model)
    
    def get_posi_angle_vec(position):
        return [cal_angle(position, i) for i in range(d_model)]
    
    sinusoid_table = np.array([get_posi_angle_vec(pos_i) for pos_i in range(pos)])
    
    # 배열의 짝수 인덱스에 사인(sin)을 적용합니다: 2i
    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])
    # 배열의 홀수 인덱스에 코사인(cos)을 적용합니다: 2i+1
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])
    
    return sinusoid_table

### 2) 마스크 생성

In [40]:
def generate_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    # 패딩을 넣기 위해 어텐션 로짓(logit)에 추가적인 차원을 넣습니다.
    return seq[:, tf.newaxis, tf.newaxis, :]   # (batch_size, 1, 1, seq_len)

# 향후 토큰을 순차적으로 마스킹하는 데 사용됩니다. 
# 즉, 마스크는 사용할 수 없는 항목을 나타냅니다.
def generate_causality_mask(src_len, tgt_len):
    mask = 1 - np.cumsum(np.eye(src_len, tgt_len), 0)
    # np.cumsum(): 배열에서 행에 따라 누적되는 원소들의 누적합 계산
    # np.eye(): 대각선이 1인 seq_len x seq_len 크기의 대각행렬 생성
    return tf.cast(mask, tf.float32)   # tf.cast: mask(텐서)를 float32로 변환

def generate_masks(src, tgt):
    enc_mask = generate_padding_mask(src)
    dec_mask = generate_padding_mask(tgt)
    
    dec_causality_mask = generate_causality_mask(tgt.shape[1], tgt.shape[1])
    dec_mask = tf.maximum(dec_mask, dec_causality_mask)
    
    dec_enc_causality_mask = generate_causality_mask(tgt.shape[1], src.shape[1])
    dec_enc_mask = tf.maximum(enc_mask, dec_enc_causality_mask)
    
    return enc_mask, dec_enc_mask, dec_mask

In [41]:
x = tf.constant([[7, 6, 0, 0, 1], [1, 2, 3, 0, 0], [0, 0, 0, 4, 5]])
generate_padding_mask(x)

<tf.Tensor: shape=(3, 1, 1, 5), dtype=float32, numpy=
array([[[[0., 0., 1., 1., 0.]]],


       [[[0., 0., 0., 1., 1.]]],


       [[[1., 1., 1., 0., 0.]]]], dtype=float32)>

In [42]:
x = tf.random.uniform((1, 3))
temp = generate_causality_mask(x.shape[0], x.shape[1])
print(x)
print(temp)

tf.Tensor([[0.12119007 0.00091314 0.81846964]], shape=(1, 3), dtype=float32)
tf.Tensor([[0. 1. 1.]], shape=(1, 3), dtype=float32)


### 3) Multi-head Attention

In [43]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        
        self.depth = d_model // self.num_heads
        
        self.W_q = tf.keras.layers.Dense(d_model)
        self.W_k = tf.keras.layers.Dense(d_model)
        self.W_v = tf.keras.layers.Dense(d_model)
        
        self.linear = tf.keras.layers.Dense(d_model)
        
        
    def scaled_dot_product_attention(self,Q,K,V,mask):
        ''' 어텐션 가중치 계산 '''
        
        d_k = tf.cast(K.shape[-1], tf.float32)
        QK = tf.matmul(Q, K, transpose_b=True)  # (..., seq_len_q, seq_len_k)
        
        scaled_qk = QK / tf.math.sqrt(d_k)   # QK를 스케일링
        
        # 스케일링된 텐서에 마스크를 더한다.
        if mask is not None: scaled_qk += (mask * -1e9)
        
        # 소프트맥스의 마지막 축을 정규화하여 스코어의 합이 1이 되도록 만듬
        attentions = tf.nn.softmax(scaled_qk, axis=-1)  # (..., seq_len_q, seq_len_k)
        out = tf.matmul(attentions, V)    # 어텐션 가중치와 V(값) 벡터의 곱셈
                                    # 포커스하고자 하는 단어가 그대로 유지되고 
                                    # 관련 없는 단어가 지워짐
        
        return out, attentions
    
    
    def split_heads(self,x):
        bsz = x.shape[0]  # batch size
        # reshape - shape의 한 원소만 -1, 
        # 의미는 전체 크기가 일정하게 유지되도록 해당 차원의 길이가 자동으로 계산
        split_x = tf.reshape(x, (bsz, -1, self.num_heads, self.depth))
        split_x = tf.transpose(split_x, perm=[0, 2, 1, 3])  #  perm은 치환하는 위치를 알려줌
        
        return split_x
        
    def combine_heads(self, x):     
        bsz = x.shape[0]
        combined_x = tf.transpose(x, perm=[0, 2, 1, 3])
        combined_x = tf.reshape(combined_x, (bsz, -1, self.d_model))

        return combined_x


    def call(self, Q, K, V, mask):
        # Linear 레이어 추가 - embedding 매핑
        WQ = self.W_q(Q)
        WK = self.W_k(K)
        WV = self.W_v(V)

        WQ_splits = self.split_heads(WQ)
        WK_splits = self.split_heads(WK)
        WV_splits = self.split_heads(WV)

        out, attention_weights = self.scaled_dot_product_attention(
            WQ_splits, WK_splits, WV_splits, mask)

        out = self.combine_heads(out)
        out = self.linear(out)

        return out, attention_weights

In [44]:
'''
스케일드 닷-프로덕트 어텐션 테스트
'''

# def print_out(q, k, v):
#     temp_out, temp_attn = scaled_dot_product_attention(
#       q, k, v, None)
#     print ('어텐션 가중치:')
#     print (temp_attn)
#     print ('출력 값:')
#     print (temp_out)

# ---------------
# np.set_printoptions(suppress=True)

# temp_k = tf.constant([[10,0,0],
#                       [0,10,0],
#                       [0,0,10],
#                       [0,0,10]], dtype=tf.float32)  # (4, 3)

# temp_v = tf.constant([[   1,0],
#                       [  10,0],
#                       [ 100,5],
#                       [1000,6]], dtype=tf.float32)  # (4, 2)

# temp_q = tf.constant([[0, 10, 0]], dtype=tf.float32)  # (1, 3)
# print_out(temp_q, temp_k, temp_v)
    
#     -----------------------
# temp_q = tf.constant([[0, 0, 10]], dtype=tf.float32)  # (1, 3)
# print_out(temp_q, temp_k, temp_v)

# --------------
# temp_q = tf.constant([[10, 10, 0]], dtype=tf.float32)  # (1, 3)
# print_out(temp_q, temp_k, temp_v)
# ---------------------
# temp_q = tf.constant([[0, 0, 10], [0, 10, 0], [10, 10, 0]], dtype=tf.float32)  # (3, 3)
# print_out(temp_q, temp_k, temp_v)
    

'\n스케일드 닷-프로덕트 어텐션 테스트\n'

### 4) Position-wise Feed Forward Network

In [45]:
class PoswiseFeedForwardNet(tf.keras.layers.Layer):
    def __init__(self,d_model,d_ff):
        super(PoswiseFeedForwardNet, self).__init__()
        self.d_model = d_model
        self.d_ff = d_ff
        
        self.fc1 = tf.keras.layers.Dense(d_ff, activation='relu')
        self.fc2 = tf.keras.layers.Dense(d_model)
        
    def call(self,x):
        out = self.fc1(x)
        out = self.fc2(out)
        
        return out  
    

### 5) Encoder Layer

In [46]:
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self,d_model,n_heads,d_ff,dropout):
        super(EncoderLayer, self).__init__()
        
        self.enc_self_attn = MultiHeadAttention(d_model, n_heads)
        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)
        
        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        
        self.do = tf.keras.layers.Dropout(dropout)
        
    def call(self,x,mask):
        
        '''
        Multi-Head Attention
        '''
        residual = x
        out = self.norm_1(x)
        out, enc_attn = self.enc_self_attn(out, out, out, mask)
        out = self.do(out)
        out += residual
        
        '''
        Position-Wise Feed Forward Network
        '''
        residual = out
        out = self.norm_2(out)
        out = self.ffn(out)
        out = self.do(out)
        out += residual
        
        return out, enc_attn

### 6) Decoder Layer

In [47]:
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self,d_model,num_heads,d_ff,dropout):
        super(DecoderLayer, self).__init__()
        
        self.dec_self_attn = MultiHeadAttention(d_model, num_heads)
        self.enc_dec_attn = MultiHeadAttention(d_model, num_heads)
        
        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)
        
        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        
        self.do = tf.keras.layers.Dropout(dropout)
        
    def call(self,x,enc_out,causality_mask,padding_mask):
        
        '''
        Masked Multi-Head Attention
        '''
        residual = x
        out = self.norm_1(x)
        out, dec_attn = self.dec_self_attn(out, out, out, padding_mask)
        out = self.do(out)
        out += residual
        
        '''
        Multi-Head Attention
        '''
        residual = out
        out = self.norm_2(out)
        out, dec_enc_attn = self.dec_self_attn(out, enc_out, enc_out, causality_mask)
        out = self.do(out)
        out += residual
        
        '''
        Position-Wise Feed Forward Network
        '''
        residual = out
        out = self.norm_3(out)
        out = self.ffn(out)
        out = self.do(out)
        out += residual
        
        return out, dec_attn, dec_enc_attn

### 7) Encoder

In [48]:
class Encoder(tf.keras.Model):
    def __init__(self,
                n_layers,
                d_model,
                n_heads,
                d_ff,
                dropout):
        super(Encoder, self).__init__()
        self.n_layers = n_layers
        self.enc_layers = [EncoderLayer(d_model, n_heads, d_ff, dropout)
                          for _ in range(n_layers)]
        self.do = tf.keras.layers.Dropout(dropout)
        
    def call(self,x,mask):
        out = x
        
        enc_attns = list()
        for i in range(self.n_layers):
            out, enc_attn = self.enc_layers[i](out, mask)
            enc_attns.append(enc_attn)
            
        return out, enc_attns

### 8) Decoder

In [49]:
class Decoder(tf.keras.Model):
    def __init__(self,
                n_layers,
                d_model,
                n_heads,
                d_ff,
                dropout):
        super(Decoder, self).__init__()
        self.n_layers = n_layers
        self.dec_layers = [DecoderLayer(d_model, n_heads, d_ff, dropout)
                          for _ in range(n_layers)]
        
    def call(self,x,enc_out,causality_mask,padding_mask):
        out = x
        
        dec_attns = list()
        dec_enc_attns = list()
        for i in range(self.n_layers):
            out, dec_attn, dec_enc_attn = self.dec_layers[i](out, enc_out,
                                                            causality_mask,
                                                            padding_mask)
            
            dec_attns.append(dec_attn)
            dec_enc_attns.append(dec_enc_attn)
            
        return out, dec_attns, dec_enc_attns

### 9) Transformer 전체 모델 조립

In [50]:
class Transformer(tf.keras.Model):
    def __init__(self,
                n_layers,
                d_model,
                n_heads,
                d_ff,
                src_vocab_size,
                tgt_vocab_size,
                pos_len,
                dropout=0.2,
                shared_fc=True,
                shared_emb=False):
        super(Transformer, self).__init__()
        
        self.d_model = tf.cast(d_model, tf.float32)
        
        if shared_emb:
            self.enc_emb = self.dec_emb = tf.keras.layers.Embedding(src_vocab_size, d_model)
        else:
            self.enc_emb = tf.keras.layers.Embedding(src_vocab_size, d_model)
            self.dec_emb = tf.keras.layers.Embedding(tgt_vocab_size, d_model)
            
        self.pos_encoding = positional_encoding(pos_len, d_model)
        self.do = tf.keras.layers.Dropout(dropout)
        
        self.encoder = Encoder(n_layers, d_model, n_heads, d_ff, dropout)
        self.decoder = Decoder(n_layers, d_model, n_heads, d_ff, dropout)
        
        self.fc = tf.keras.layers.Dense(tgt_vocab_size)
        
        self.shared_fc = shared_fc
        
        if shared_fc:
            self.fc.set_weights(tf.transpose(self.dec_emb.weights))
            
    def embedding(self,emb,x):
        seq_len = x.shape[1]
        
        out = emb(x)
        
        if self.shared_fc: out *= tf.math.sqrt(self.d_model)
            
        out += self.pos_encoding[np.newaxis, ...][:, :seq_len, :]
        out = self.do(out)
        
        return out
    
    def call(self,enc_in,dec_in,enc_mask,causality_mask,dec_mask):
        enc_in = self.embedding(self.enc_emb, enc_in)
        dec_in = self.embedding(self.dec_emb, dec_in)
        
        enc_out, enc_attns = self.encoder(enc_in, enc_mask)
        
        dec_out, dec_attns, dec_enc_attns = self.decoder(dec_in, enc_out,
                                                        causality_mask, dec_mask)
        
        logits = self.fc(dec_out)
        
        return logits, enc_attns, dec_attns, dec_enc_attns       

### 10) 모델 인스턴스 생성

In [51]:
VOCAB_SIZE = 20000
d_model = 368

transformer = Transformer(n_layers=1,
                         d_model=d_model,
                         n_heads=8,
                         d_ff=1024,
                         src_vocab_size=VOCAB_SIZE,
                         tgt_vocab_size=VOCAB_SIZE,
                         pos_len=200,
                         dropout=0.2,
                         shared_fc=True,
                         shared_emb=True)


### 11) Learning Rate Scheduler

In [52]:
# Learning Rate Scheduler구현
class LearningRateScheduler(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(LearningRateScheduler, self).__init__()
        
        self.d_model = d_model
        self.warmup_steps = warmup_steps
        
    def __call__(self, step):
        arg1 = step ** -0.5
        arg2 = step * (self.warmup_steps ** -1.5)
        
        return (self.d_model ** -0.5) * tf.math.minimum(arg1, arg2)
        

### 12) Learning Rate & Optimizer

In [53]:
# Learning Rate 인스턴스 선언 & Optimizer 구현
learning_rate = LearningRateScheduler(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate,
                                    beta_1=0.9,
                                    beta_2=0.98,
                                    epsilon=1e-9)
print("슝=3")

슝=3


### 13) Loss Function 정의

In [54]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,
                                                           reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)
    
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    
    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)

### 14) Train Step 정의

In [55]:
@tf.function()
def train_step(src,tgt,model,optimizer):
    tgt_in = tgt[:, :-1]   # Decoder의 input
    gold = tgt[:, 1:]     # Decoder의 output과 비교하기 위해 rihgt shift를 통해  생성한 최종 타겟
    
    enc_mask, dec_enc_mask, dec_mask = generate_masks(src, tgt_in)
    
    with tf.GradientTape() as tape:
        predictions, enc_attns, dec_attns, dec_enc_attns = model(src, tgt_in, enc_mask,
                                                                dec_enc_mask, dec_mask)
        loss = loss_function(gold, predictions)
        
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    return loss, enc_attns, dec_attns, dec_enc_attns

## (2) 훈련을 시키자

[예문]   
1. 지루하다, 놀러가고 싶어.   
2. 오늘 일찍 일어났더니 피곤하다.   
3. 간만에 여자친구랑 데이트 하기로 했어.   
4. 집에 있는다는 소리야.   

---   

[제출Translations]   
> 1. 잠깐 쉬 어도 돼요 . \<end>   
> 2. 맛난 거 드세요 . \<end>   
> 3. 떨리 겠 죠 . \<end>   
> 4. 좋 아 하 면 그럴 수 있 어요 . \<end>   
    
[Hyperparameters]    
> n_layers: 1   
> d_model: 368    
> n_heads: 8    
> d_ff: 1024   
> dropout: 0.2   
   
[Training Parameters]   
> Warmup Steps: 1000   
> Batch Size: 64   
> Epoch At: 10   

In [56]:
def get_decoded_sentence(encoded_sentence, idx2word):
    return ' '.join(idx2word[index] if index in idx2word else '<UNK>' for index in encoded_sentence[1:]) 


def get_decoded_sentences(encoded_sentences, idx2word):
    return [get_decoded_sentence(encoded_sentence, idx2word) for encoded_sentence in encoded_sentences]

In [ ]:
# def evaluate(sentence, model, src_tokenizer, tgt_tokenizer):
#     sentence = preprocess_sentence(sentence)
    
#     pieces = src_tokenizer.encode_as_pieces(sentence) # 문자열을 token으로 분할
#     tokens = src_tokenizer.encode_as_ids(sentence) # 문자열을 숫자로 분할
    
#     _input = tf.keras.preprocessing.sequence.pad_sequences([tokens],
#                                                           maxlen=enc_train.shape[-1],
#                                                           padding='post')
    
#     ids = []
#     output = tf.expand_dims([tgt_tokenizer.bos_id()], 0)
#     for i in range(dec_train.shape[-1]):
#         enc_padding_mask, combined_mask, dec_padding_mask = generate_masks(_input, output)
        
#         predictions, enc_attns, dec_attns, dec_enc_attns = model(_input, output,
#                                                                 enc_padding_mask,
#                                                                 combined_mask,
#                                                                 dec_padding_mask)
        
#         predicted_id = tf.argmax(tf.math.softmax(predictions, axis=-1)[0,-1]).numpy().item()
#         # predictions에 소프트맥스 함수를 적용하여 가장 큰 값의 인덱스를 predicted_id로 저장
        
#         if tgt_tokenizer.eos_id() == predicted_id:
#             result = tgt_tokenizer.decode_ids(ids) # 숫자를 문자열로 복원
#             return pieces, result, enc_attns, dec_attns, dec_enc_attns
        
#         ids.append(predicted_id)
#         output = tf.concat([output, tf.expand_dims([predicted_id], 0)], axis=-1)
        
#     result = tgt_tokenizer.decode_ids(ids)
#     return pieces, result, enc_attns, dec_attns, dec_enc_attns
  

In [ ]:
# def translate(sentence, model, src_tokenizer, tgt_tokenizer):
#     pieces, result, enc_attns, dec_attns, dec_enc_attns = evaluate(sentence, model,
#                                                                   src_tokenizer, tgt_tokenizer)
    
#     return result

In [57]:
# translate()

def evaluate(sentence, model):
    # sentence 전처리(enc_train과 같은 모양으로)
    sentence = preprocess_sentence(sentence)
    pieces = sentence
    tokens = get_encoded_sentence(pieces, word_to_index)

    _input = tf.keras.preprocessing.sequence.pad_sequences([tokens],
                                                           maxlen=enc_train.shape[-1],
                                                           padding='post')
    
    ids = []
    
    output = tf.expand_dims([word_to_index["<start>"]], 0) 
    for i in range(dec_train.shape[-1]):
        enc_padding_mask, combined_mask, dec_padding_mask = \
        generate_masks(_input, output)

        predictions, enc_attns, dec_attns, dec_enc_attns =\
        model(_input, 
              output,
              enc_padding_mask,
              combined_mask,
              dec_padding_mask)

        predicted_id = \
        tf.argmax(tf.math.softmax(predictions, axis=-1)[0, -1]).numpy().item()
        
        # 예측 단어가 종료 토큰일 경우
        if word_to_index["<end>"] == predicted_id:
            result = get_decoded_sentence(ids, index_to_word)
            return pieces, result, enc_attns, dec_attns, dec_enc_attns
        ##word_to_index
        ids.append(predicted_id)
        output = tf.concat([output, tf.expand_dims([predicted_id], 0)], axis=-1)

    result = get_decoded_sentence(ids, index_to_word)

    return pieces, result, enc_attns, dec_attns, dec_enc_attns

def translate(sentence, model):
    pieces, result, enc_attns, dec_attns, dec_enc_attns = \
    evaluate(sentence, model)
    
    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))

    return result

In [58]:
examples = [
    "지루하다, 놀러가고 싶어.",
    "오늘 일찍 일어났더니 피곤하다.",
    "간만에 여자친구랑 데이트 하기로 했어.",
    "집에 있는다는 소리야."
]

In [59]:
# from tqdm import tqdm_notebook
from tqdm.notebook import tqdm  # 버전 5.0 이상

BATCH_SIZE = 64
EPOCHS = 10

for epoch in range(EPOCHS):
    total_loss = 0
    
    idx_list = list(range(0, enc_train.shape[0], BATCH_SIZE))
    random.shuffle(idx_list)
    # t = tqdm_notebook(idx_list)   
    t = tqdm(idx_list)    # 버전 5.0 이상
    
    for (batch, idx) in enumerate(t):
        batch_loss, enc_attns, dec_attns, dec_enc_attns = train_step(enc_train[idx:idx+BATCH_SIZE],
                                                                     dec_train[idx:idx+BATCH_SIZE],
                                                                     transformer,
                                                                     optimizer)
        
        total_loss += batch_loss
        
        t.set_description_str(f'Epoch {epoch + 1}')
        t.set_postfix_str(f'Loss{total_loss.numpy() / (batch + 1):.4f}')

  0%|          | 0/519 [00:00<?, ?it/s]

  0%|          | 0/519 [00:00<?, ?it/s]

  0%|          | 0/519 [00:00<?, ?it/s]

  0%|          | 0/519 [00:00<?, ?it/s]

  0%|          | 0/519 [00:00<?, ?it/s]

  0%|          | 0/519 [00:00<?, ?it/s]

  0%|          | 0/519 [00:00<?, ?it/s]

  0%|          | 0/519 [00:00<?, ?it/s]

  0%|          | 0/519 [00:00<?, ?it/s]

  0%|          | 0/519 [00:00<?, ?it/s]

In [60]:
print("Translations")   
for example in examples:
    translate(example, transformer)

Translations
Input: 지루하다, 놀러가고 싶어.
Predicted translation: 생각 보다 나 봐요 .
Input: 오늘 일찍 일어났더니 피곤하다.
Predicted translation: 노동 을 했 나 봐요 .
Input: 간만에 여자친구랑 데이트 하기로 했어.
Predicted translation: 이 많 은 것 같 아요 .
Input: 집에 있는다는 소리야.
Predicted translation: 은 알 게 된 관계 가 좋 아 질 거 예요 .


# Step7. 성능 측정하기(BLEU Score)
- 챗봇의 경우, 올바른 대댭을 하는지가 중요한 평가지표다. 이를 위한 BLEU Score를 계산하는 calculate_bleu()함수를 적용한다.

### 1) NLTK를 활용한 BLEU Score

In [61]:
from nltk.translate.bleu_score import sentence_bleu

reference = '많 은 자연어 처리 연구자 들 이 트랜스포머 를 선호 한다.'.split()
candidate = '적 은 자연어 학 개발자 들 가 트랜스포머 을 선호 한다 요'.split()

print(f'원문 : {reference}')
print(f'번역문 : {candidate}')
print(f'BLEU Score : {sentence_bleu([reference], candidate)}')

원문 : ['많', '은', '자연어', '처리', '연구자', '들', '이', '트랜스포머', '를', '선호', '한다.']
번역문 : ['적', '은', '자연어', '학', '개발자', '들', '가', '트랜스포머', '을', '선호', '한다', '요']
BLEU Score : 6.5806869883189804e-155


/opt/conda/lib/python3.7/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/opt/conda/lib/python3.7/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


$(\prod_{i=1}^4 precision_i)^{\frac{1}{4}} = (\text{1-gram} \times\text{2-gram} \times\text{3-gram} \times\text{4-gram})^{\frac{1}{4}}$

BLEU Score의 정의로 돌아가 한번 따져봅시다. BLEU Score가 N-gram으로 점수를 측정한다는 것을 기억하실 거예요. 아래 수식을 기억하시죠? 1-gram부터 4-gram까지의 점수(Precision)를 모두 곱한 후, 루트를 두 번 씌우면$(^{1/4})$ BLEU Score가 된답니다. 진정 멋진 번역이라면, 모든 N-gram에 대해서 높은 점수를 얻었을 거예요. 그렇다면 위에서 살펴본 예시에서는 각 N-gram이 점수를 얼마나 얻었는지 확인해보도록 합시다. weights의 디폴트값은 [0.25, 0.25, 0.25, 0.25]로 1-gram부터 4-gram까지의 점수에 가중치를 동일하게 주는 것이지만, 만약 이 값을 [1, 0, 0, 0]으로 바꿔주면 BLEU Score에 1-gram의 점수만 반영하게 됩니다.

In [62]:
print(f'1-gram : {sentence_bleu([reference], candidate, weights=[1, 0, 0, 0])}')
print(f'2-gram : {sentence_bleu([reference], candidate, weights=[0, 1, 0, 0])}')
print(f'3-gram : {sentence_bleu([reference], candidate, weights=[0, 0, 1, 0])}')
print(f'4-gram : {sentence_bleu([reference], candidate, weights=[0, 0, 0, 1])}')

1-gram : 0.4166666666666667
2-gram : 0.0909090909090909
3-gram : 2.2250738585072626e-308
4-gram : 2.2250738585072626e-308


- BLEU 계산시 특정 N-gram이 0점이 나와서 BLEU가 너무 커지거나 작아지는 쪽으로 왜곡되는 문제를 보완하기 위해 SmoothingFunction() 을 사용하고 있습니다. Smoothing 함수는 모든 Precision에 아주 작은 epsilon 값을 더해주는 역할을 하는데, 이로써 0점이 부여된 Precision도 완전한 0이 되지 않으니 점수를 1.0 으로 대체할 필요가 없어지죠. 즉, 우리의 의도대로 점수가 계산되는 거예요.
   
- 진실된 BLEU Score를 확인하기 위해 어서 SmoothingFunction() 을 적용해봅시다! 아래 코드에서는 SmoothingFunction().method1을 사용해 보겠습니다. 자신만의 Smoothing 함수를 구현해서 적용할 수도 있겠지만, nltk에서는 method0부터 method7까지를 이미 제공하고 있습니다.

### 2) SmoothingFunction()으로 BLEU Score보정하기
- BLEU 계산시 특정 N-gram이 0점이 나와서 BLEU가 너무 커지거나 작아지는 쪽으로 왜곡되는 문제를 보완하기 위해 SmoothingFunction() 을 사용
- Smoothing 함수는 모든 Precision에 아주 작은 epsilon 값을 더해주는 역할을 하는데, 이로써 0점이 부여된 Precision도 완전한 0이 되지 않으니 점수를 1.0 으로 대체할 필요가 없어지죠. 즉 우리의 의도대로 점수가 계산되는 거예요.

In [63]:
from nltk.translate.bleu_score import SmoothingFunction

def calculate_bleu(reference, candidate, weights=[0.25, 0.25, 0.25, 0.25]):
    return sentence_bleu([reference], candidate, weights=weights,
                        smoothing_function=SmoothingFunction().method1)

print(f'BLEU-1 : {calculate_bleu(reference, candidate, weights=[1, 0, 0, 0])}')
print(f'BLEU-2 : {calculate_bleu(reference, candidate, weights=[0, 1, 0, 0])}')
print(f'BLEU-3 : {calculate_bleu(reference, candidate, weights=[0, 0, 1, 0])}')
print(f'BLEU-4 : {calculate_bleu(reference, candidate, weights=[0, 0, 0, 1])}')

print(f'\nBLEU-total : {calculate_bleu(reference, candidate)}')

BLEU-1 : 0.4166666666666667
BLEU-2 : 0.0909090909090909
BLEU-3 : 0.010000000000000004
BLEU-4 : 0.011111111111111112

BLEU-total : 0.045293761707938834


### 3) 트랜스포머 모델의 번역 성능 알아보기

- 여기서 BLEU-4가 BLEU-3보다 약간이나마 점수가 높은 이유는 한 문장에서 발생하는 3-gram 쌍의 개수와 4-gram 쌍의 개수를 생각해보면 이해할 수 있습니다. 각 Precision을 N-gram 개수로 나누는 부분에서 차이가 발생하는 것이죠.

In [64]:
def eval_bleu(src_corpus, tgt_corpus, verbose=True):
    total_score = 0.0
    sample_size = len(tgt_corpus)

    for idx in tqdm_notebook(range(sample_size)):
        src_tokens = src_corpus[idx]
        tgt_tokens = tgt_corpus[idx]
        
        src = []
        tgt = []
        
        for word in src_tokens:
            if word !=0 and word !=1 and word !=3 and word !=4:
                src.append(word)
        
        for word in tgt_tokens:
            if word != 0 and word != 3 and word !=4:
                tgt.append(word)

        src_sentence = get_decoded_sentence(src, index_to_word)
        tgt_sentence = get_decoded_sentence(tgt, index_to_word)
        
        
        reference = preprocess_sentence(tgt_sentence)
        candidate = translate(src_sentence, transformer)

        score = sentence_bleu([reference], candidate,
                              smoothing_function=SmoothingFunction().method1)
        total_score += score

        if verbose:
            print("Source Sentence: ", src_sentence)
            print("Model Prediction: ", candidate)
            print("Real: ", reference)
            print("Score: %lf\n" % score)

    print("Num of Sample:", sample_size)
    print("Total Score:", total_score / sample_size)

In [65]:
eval_bleu(enc_val[::5], dec_val[::5], verbose=True)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/68 [00:00<?, ?it/s]

Input: 하 네
Predicted translation: 세요 .
Source Sentence:  하 네
Model Prediction:  세요 .
Real:  ['이', '복잡', '한가요', '.']
Score: 0.080343

Input: 미리 걱정 해
Predicted translation: 은 먼저 사랑 을 나가 세요 .
Source Sentence:  미리 걱정 해
Model Prediction:  은 먼저 사랑 을 나가 세요 .
Real:  ['데', '없', '는', '걱정', '일', '수', '도', '있', '어요', '.']
Score: 0.011503

Input: 가 극 에 달했었 는데
Predicted translation: 은 괜찮 아 졌 길 바랄게요 .
Source Sentence:  가 극 에 달했었 는데
Model Prediction:  은 괜찮 아 졌 길 바랄게요 .
Real:  ['은', '좀', '괜찮', '아', '졌', '길', '바랄게요', '.']
Score: 0.017201

Input: 커피 한잔을
Predicted translation: 시간 이 라면 저 도 필요 해요 .
Source Sentence:  커피 한잔을
Model Prediction:  시간 이 라면 저 도 필요 해요 .
Real:  ['면', '좋', '죠', '.']
Score: 0.012109

Input: 주 에 혼자 여행길 가려 하 는데
Predicted translation: 은 생각 이 에요 . 혼자 안 좋 은 것 이 힘들 었 겠 어요 .
Source Sentence:  주 에 혼자 여행길 가려 하 는데
Model Prediction:  은 생각 이 에요 . 혼자 안 좋 은 것 이 힘들 었 겠 어요 .
Real:  ['은', '생각', '이', '에요', '.', '혼자', '하', '는', '여행', '은', '기분', '전환', '에', '도움', '이', '돼요', '.']
Score: 0.008388

Input: 해도 

Input: 운 이 안 좋 었 어
Predicted translation: 곳은 에서 상담 을 받 아 보 세요 .
Source Sentence:  운 이 안 좋 었 어
Model Prediction:  곳은 에서 상담 을 받 아 보 세요 .
Real:  ['곳', '에', '쓰', '려고', '운', '을', '아껴', '뒀', '나', '봐요', '.']
Score: 0.012918

Input: 한 지 10 일 차 점점 나아지 며 있 습니다 .
Predicted translation: 도 인연 이 라면 자연 스럽 게 기본 이 다를 거 예요 .
Source Sentence:  한 지 10 일 차 점점 나아지 며 있 습니다 .
Model Prediction:  도 인연 이 라면 자연 스럽 게 기본 이 다를 거 예요 .
Real:  ['랑', '대화', '하', '는', '게', '위', '로', '가', '되', '었', '으면', '합니다', '.']
Score: 0.006938

Input: 인데 그냥 취직 할까 ?
Predicted translation: 나 할 수 있 을 거 예요 .
Source Sentence:  인데 그냥 취직 할까 ?
Model Prediction:  나 할 수 있 을 거 예요 .
Real:  ['하', '는', '것', '자체', '가', '중요', '하', '죠', '.']
Score: 0.012301

Input: 는 그대로 의 나 를 사랑 하 기
Predicted translation: 사랑 이 자 면 좋 이 이 이 이 이 이 이 가 사랑 이 더 사랑 이 에요 .
Source Sentence:  는 그대로 의 나 를 사랑 하 기
Model Prediction:  사랑 이 자 면 좋 이 이 이 이 이 이 이 가 사랑 이 더 사랑 이 에요 .
Real:  ['사람', '이', '자', '멋진', '말', '이', '네요', '또한']
Score: 0.005641

Input: 친구 가 잠 이 너무 많 아 .
Predicted t

# 성과평가 및 회고

## (1) 성과평가
- 챗봇 훈련데이터를 위한 전처리와 데이터증식으로 원데이터 11,823개에서 증식데이터 21,797개를 늘렸다.
- 과제에서 제시된 하이퍼파라미터를 적용하여 에폭은 10회로 적용하고, 에폭마다 소요 시간은 1분 30초이고 Loss는 5.6231에서 시작해 0.4229로 줄어들었다.
- 68문장에 대해 BLEU평가한 결과 0.015971131389754555로 매우 낮다.
- 주어가 대부분 사라져 문장들이 불완전하다. 아마 위키의 단어장이어서 한국어의 특징을 제대로 반영하지 못한듯핟.

## (2) 회고
- 9장 내용에 BLEU라는 개념이 갑자기 나와, 이것 또한 뭐지하면서 위키백과를 찾아봤다. 자연어에서의 성과지표라는 것을 확인하고 따로 개념을 챙겼었다. 이번에 좀더 깊이 있는 내용을 접할 수 있었지만, 기본 개념만 챙겼다. 그런데 가만히 생각해보면 문장간에 얼마나 순서가 동일한 단어들이 배열되어 있냐는 건데....그래서인지 높은 점수가 나올 수는 없다. 의미상으로 체크할 수 있는 방법은? 이거 대단한 발견이겠는걸....
- ko.bin을 로드 하는 데 실패를 반복하여 시간을 많이 소비되었다. 아쉬운 것은 결국 해결을 하지 못했다는 거다. 다급하게 위키의 한국 단어집을 하용하였다.
- 아직까지 BoW를 만들고 벡터화하는 전체 프로세스를 정확히 파악하지 모하고 있다. 흐름도를 정립하여 다른 작업마다 혼돈이 없도록 해야겠다.
- CV와 LNP의 근본적인 차이는 CV의 경우에는 원천데이타가 이미 벡터화되어 있다는 거다. 반면 NLP는 다양한 방법으로 벡터화할 수 있다. 앞단에서 이것을 어떻게 설정하느냐가 관건이다. 아직까지 각각의 차이와 성과와의 연결이 되지 않고 있다. 관련 동영상을 보면서 정립해야겠다.